### Import and Install Libraries

In [1]:
!pip install sqlalchemy
!pip install pymysql

In [92]:
import requests
import pandas as pd
from IPython.display import JSON
import my_stuff

### Scrapped Data


In [4]:
city_df ={'city':['Berlin','London'] ,"country":['Germany', 'United Kingdom'],"population": [3755251,8799800],"latitude":['52°31′12″N','51°30′26″N'], "longitude": ['13°24′18″E','0°7′39″W']}
city_df= pd.DataFrame(city_df)

### Notebook to SQL

In [6]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password =my_stuff.my_sql_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [7]:
#city_df.to_sql('city', if_exists ='append', con =connection_string, index = False)

In [8]:
city_from_sql=pd.read_sql('city', con=connection_string)

In [9]:
city_from_sql

,city_id,city,country,population,latitude,longitude
0,1,Berlin,Germany,3755251,52°31′12″N,13°24′18″E
1,2,London,United Kingdom,8799800,51°30′26″N,0°7′39″W


In [10]:
airport_df = {'ICAO':['EDDB','EGLL'] ,"city_id":[1, 2]}
airport_df= pd.DataFrame(airport_df)

In [12]:
#airport_df.to_sql('airport', if_exists ='append', con =connection_string, index = False)

In [13]:
airport_df = pd.read_sql('airport', con= connection_string)
airport_df

,icao,city_id
0,EDDB,1
1,EGLL,2


## Weather Details

#### API Calls

In [15]:
Response= requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={'Berlin'}&appid={'9c0f851b94d1cae29abf8a85c9c458f7'}&units=metric")

In [16]:
Response_= Response.json()
Response_

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1715180400,
   'main': {'temp': 18.13,
    'feels_like': 17.1,
    'temp_min': 17.36,
    'temp_max': 18.13,
    'pressure': 1014,
    'sea_level': 1014,
    'grnd_level': 1019,
    'humidity': 42,
    'temp_kf': 0.77},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}],
   'clouds': {'all': 33},
   'wind': {'speed': 3.62, 'deg': 61, 'gust': 3.83},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2024-05-08 15:00:00'},
  {'dt': 1715191200,
   'main': {'temp': 16.08,
    'feels_like': 14.92,
    'temp_min': 14.86,
    'temp_max': 16.08,
    'pressure': 1020,
    'sea_level': 1020,
    'grnd_level': 1020,
    'humidity': 45,
    'temp_kf': 1.22},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}],
   'clouds': {'all': 49},
   'wind': {'speed': 3.04, 'deg': 70, 'gust': 5},
   'visibility

In [17]:
JSON(Response_)

<IPython.core.display.JSON object>

#### Preliminary

In [18]:
Response_['list'][0]

{'dt': 1715180400,
 'main': {'temp': 18.13,
  'feels_like': 17.1,
  'temp_min': 17.36,
  'temp_max': 18.13,
  'pressure': 1014,
  'sea_level': 1014,
  'grnd_level': 1019,
  'humidity': 42,
  'temp_kf': 0.77},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'clouds': {'all': 33},
 'wind': {'speed': 3.62, 'deg': 61, 'gust': 3.83},
 'visibility': 10000,
 'pop': 0,
 'sys': {'pod': 'd'},
 'dt_txt': '2024-05-08 15:00:00'}

In [19]:
Response_['list'][0]['main']['temp']

18.13

In [20]:
Response_['list'][0]['main']['humidity']

42

In [21]:
Response_['list'][0]['pop']

0

In [22]:
Response_['list'][0]['weather'][0]['main']

'Clouds'

In [23]:
Response_['list'][0]['wind']['speed']

3.62

In [24]:
Response_['list'][0]

{'dt': 1715180400,
 'main': {'temp': 18.13,
  'feels_like': 17.1,
  'temp_min': 17.36,
  'temp_max': 18.13,
  'pressure': 1014,
  'sea_level': 1014,
  'grnd_level': 1019,
  'humidity': 42,
  'temp_kf': 0.77},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'clouds': {'all': 33},
 'wind': {'speed': 3.62, 'deg': 61, 'gust': 3.83},
 'visibility': 10000,
 'pop': 0,
 'sys': {'pod': 'd'},
 'dt_txt': '2024-05-08 15:00:00'}

#### Looping through

In [25]:
city= 'Berlin'
weather_information= {"city":[],
                "Time_stamp":[],
                "Temperature":[],
                "Humidity":[],
                "Outlook": [],
                "Rain_probability":[],
                "Wind_speed":[]}

for info in Response_["list"]:
    weather_information["city"].append(city)
    weather_information["Time_stamp"].append(info.get("dt_txt", None)),
    weather_information["Humidity"].append(info["main"].get("humidity", None))
    weather_information["Temperature"].append(info["main"].get("temp", None))
    weather_information["Outlook"].append(info["weather"][0].get("main", None))
    weather_information["Rain_probability"].append(info.get("rain", {}).get("3h", 0))
    weather_information["Wind_speed"].append(info["wind"].get("speed", None))

weather_information= pd.DataFrame(weather_information)
weather_information.head(5)




,city,Time_stamp,Temperature,Humidity,Outlook,Rain_probability,Wind_speed
0,Berlin,2024-05-08 15:00:00,18.13,42,Clouds,0.0,3.62
1,Berlin,2024-05-08 18:00:00,16.08,45,Clouds,0.0,3.04
2,Berlin,2024-05-08 21:00:00,11.33,57,Clear,0.0,2.13
3,Berlin,2024-05-09 00:00:00,9.70,64,Clear,0.0,1.66
4,Berlin,2024-05-09 03:00:00,8.38,71,Clouds,0.0,1.46


#### Weather info for 24 hours

In [ ]:
#Simply slice through the first eight entries

In [26]:
city= 'Berlin'
weather_information= {"city":[],
                "Time_stamp":[],
                "Temperature":[],
                "Humidity":[],
                "Outlook": [],
                "Rain_probability":[],
                "Wind_speed":[]}

for info in Response_["list"][:8]:#Simply slice 
    weather_information["city"].append(city)
    weather_information["Time_stamp"].append(info.get("dt_txt", None)),
    weather_information["Humidity"].append(info["main"].get("humidity", None))
    weather_information["Temperature"].append(info["main"].get("temp", None))
    weather_information["Outlook"].append(info["weather"][0].get("main", None))
    weather_information["Rain_probability"].append(info.get("rain", {}).get("3h", 0))
    weather_information["Wind_speed"].append(info["wind"].get("speed", None))

weather_information= pd.DataFrame(weather_information)
weather_information


,city,Time_stamp,Temperature,Humidity,Outlook,Rain_probability,Wind_speed
0,Berlin,2024-05-08 15:00:00,18.13,42,Clouds,0,3.62
1,Berlin,2024-05-08 18:00:00,16.08,45,Clouds,0,3.04
2,Berlin,2024-05-08 21:00:00,11.33,57,Clear,0,2.13
3,Berlin,2024-05-09 00:00:00,9.70,64,Clear,0,1.66
4,Berlin,2024-05-09 03:00:00,8.38,71,Clouds,0,1.46
5,Berlin,2024-05-09 06:00:00,11.50,57,Clouds,0,1.17
6,Berlin,2024-05-09 09:00:00,16.92,40,Clouds,0,1.21
7,Berlin,2024-05-09 12:00:00,19.86,33,Clouds,0,2.11


#### Looping for multiple cities

In [78]:
from datetime import datetime,timedelta
from pytz import timezone

berlin_timezone = timezone('Europe/Berlin')
cities=['Berlin','London']

In [28]:
weather_info= {"city":[],
                "Time_stamp":[],
                "Temperature":[],
                "Humidity":[],
                "Outlook": [],
                "Rain_probability":[],
                "Wind_speed":[],
                "Date_retrieved":[]}
for city in cities:
    url= (f"http://api.openweathermap.org/data/2.5/forecast?q={'city'}&appid={'9c0f851b94d1cae29abf8a85c9c458f7'}&units=metric")
    response= requests.get(url)
    weather_details = response.json()
    timestamp= datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
    
    for info in weather_details["list"][:8]:
        weather_info["city"].append(city)
        weather_info["Time_stamp"].append(info.get("dt_txt", None)),
        weather_info["Temperature"].append(info["main"].get("temp", None)),
        weather_info["Humidity"].append(info["main"].get("humidity", None)),
        weather_info["Outlook"].append(info["weather"][0].get("main", None)),
        weather_info["Rain_probability"].append(info.get("rain", {}).get("3h", 0)),
        weather_info["Wind_speed"].append(info["wind"].get("speed", None)),
        weather_info["Date_retrieved"].append(timestamp)

weather_information_2 =pd.DataFrame(weather_info)


In [29]:
weather_information_2.head()

,city,Time_stamp,Temperature,Humidity,Outlook,Rain_probability,Wind_speed,Date_retrieved
0,Berlin,2024-05-08 15:00:00,11.22,85,Clouds,0.00,2.14,2024-05-08 14:56:56
1,Berlin,2024-05-08 18:00:00,11.16,84,Clouds,0.00,2.18,2024-05-08 14:56:56
2,Berlin,2024-05-08 21:00:00,11.47,86,Clouds,0.00,2.08,2024-05-08 14:56:56
3,Berlin,2024-05-09 00:00:00,12.03,89,Rain,0.60,2.43,2024-05-08 14:56:56
4,Berlin,2024-05-09 03:00:00,12.66,86,Rain,0.74,2.90,2024-05-08 14:56:56


In [30]:
city_df= city_from_sql
city_df

,city_id,city,country,population,latitude,longitude
0,1,Berlin,Germany,3755251,52°31′12″N,13°24′18″E
1,2,London,United Kingdom,8799800,51°30′26″N,0°7′39″W


In [31]:
def get_connection_string():
    schema="gans"  
    host="127.0.0.1"       
    user="root"
    password=my_stuff.my_sql_password 
    port=3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    return connection_string

connect = get_connection_string()

In [95]:
def get_weather_information(city_df):
    weather_info= {"city_id":[],
                "city":[],
                "Time_stamp":[],
                "Temperature":[],
                "Humidity":[],
                "Outlook": [],
                "Rain_probability":[],
                "Wind_speed":[],
                "Date_retrieved":[]
                }
    for i,row in city_df.iterrows():#iterrows iterates through each index and row in the dataframe
        url= (f"http://api.openweathermap.org/data/2.5/forecast?q={row['city']}&appid={'9c0f851b94d1cae29abf8a85c9c458f7'}&units=metric")
        response= requests.get(url)
        weather_details = response.json()
        timestamp= datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
    
        for info in weather_details["list"][:8]:
            weather_info["city_id"].append(row["city_id"])
            weather_info["city"].append(row["city"])
            weather_info["Time_stamp"].append(info.get("dt_txt", None)),
            weather_info["Temperature"].append(info["main"].get("temp", None)),
            weather_info["Humidity"].append(info["main"].get("humidity", None)),
            weather_info["Outlook"].append(info["weather"][0].get("main", None)),
            weather_info["Rain_probability"].append(info.get("rain", {}).get("3h", 0)),
            weather_info["Wind_speed"].append(info["wind"].get("speed", None)),
            weather_info["Date_retrieved"].append(timestamp)

    weather_df =pd.DataFrame(weather_info)
    #weather_df.to_sql('weather',if_exists = 'append', con =connect, index = False)
    

In [34]:
#get_weather_information(city_df)

## Flight Details

In [35]:
import requests

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDDB/2024-05-08T12:00/2024-05-09T00:00"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"true"}

headers = {
	"X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)


In [36]:
response.status_code

200

In [37]:
flight_response= response.json()
JSON(flight_response)


<IPython.core.display.JSON object>

In [38]:
flight_response

{'departures': [{'departure': {'scheduledTime': {'utc': '2024-05-08 10:10Z',
     'local': '2024-05-08 12:10+02:00'},
    'revisedTime': {'utc': '2024-05-08 10:17Z',
     'local': '2024-05-08 12:17+02:00'},
    'runwayTime': {'utc': '2024-05-08 10:47Z',
     'local': '2024-05-08 12:47+02:00'},
    'terminal': '1',
    'checkInDesk': '514-616',
    'gate': 'A38',
    'quality': ['Basic', 'Live']},
   'arrival': {'airport': {'icao': 'LIEO', 'iata': 'OLB', 'name': 'Olbia'},
    'quality': []},
   'number': 'EC 5055',
   'callSign': 'EJU75LU',
   'status': 'Departed',
   'codeshareStatus': 'IsCodeshared',
   'isCargo': False,
   'aircraft': {'reg': 'OE-IDS', 'modeS': '440DA5', 'model': 'Airbus A320'},
   'airline': {'name': 'EasyJet Europe', 'iata': 'EC', 'icao': 'EJU'}},
  {'departure': {'scheduledTime': {'utc': '2024-05-08 10:37Z',
     'local': '2024-05-08 12:37+02:00'},
    'revisedTime': {'utc': '2024-05-08 10:37Z',
     'local': '2024-05-08 12:37+02:00'},
    'runwayTime': {'utc': '2

In [39]:
pd.json_normalize(flight_response['arrivals'])

,number,callSign,status,codeshareStatus,isCargo,departure.airport.icao,departure.airport.iata,departure.airport.name,departure.scheduledTime.utc,departure.scheduledTime.local,...,location.pressure.mmHg,location.groundSpeed.kt,location.groundSpeed.kmPerHour,location.groundSpeed.miPerHour,location.groundSpeed.meterPerSecond,location.trueTrack.deg,location.trueTrack.rad,location.reportedAtUtc,location.lat,location.lon
0,FR 1143,RYR37TH,Arrived,IsOperator,False,LPPT,LIS,Lisbon,2024-05-08 05:45Z,2024-05-08 06:45+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AY 1433,FIN50M,Arrived,IsOperator,False,EFHK,HEL,Helsinki,2024-05-08 08:25Z,2024-05-08 11:25+03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EC 5064,EJU38ZK,Arrived,IsCodeshared,False,LIML,LIN,Milan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EW 8601,EWG8J,Arrived,IsOperator,False,LPPR,OPO,Porto,2024-05-08 07:10Z,2024-05-08 08:10+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FR 1508,RYR1JM,Arrived,IsOperator,False,LECH,CDT,Castellón de la Plana,2024-05-08 07:25Z,2024-05-08 09:25+02:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,FR 5418,NaN,Expected,IsOperator,False,EIDW,DUB,Dublin,2024-05-08 18:45Z,2024-05-08 19:45+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226,I2 3674,NaN,Expected,IsOperator,False,LEMD,MAD,Madrid,2024-05-08 18:00Z,2024-05-08 20:00+02:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,IB 3674,NaN,Expected,Unknown,False,LEMD,MAD,Madrid,2024-05-08 18:00Z,2024-05-08 20:00+02:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
228,SR 6493,NaN,Expected,IsOperator,False,HEGN,HRG,Hurghada,2024-05-08 16:25Z,2024-05-08 19:25+03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
flight_response['arrivals'][0]['arrival']['scheduledTime']['local']

'2024-05-08 11:10+02:00'

In [41]:
flight_response['arrivals'][0]['departure']['airport'].get('name')

'Lisbon'

In [42]:
flight_response['arrivals'][0].get('number')

'FR 1143'

In [43]:
flight_response['arrivals'][0].get('arrival').get('terminal')

'2'

In [44]:
flight_response['arrivals'][0].get('arrival').get('gate')

'Z34'

In [45]:
flight_response['arrivals'][0].get('status')

'Arrived'

#### Looping through

In [47]:
ICAO = 'EDDB'
flight_info = {'flight_number':[],
               'departure_airpot_name': [],
               'arrival_ICAO':[],
               'arrival_terminal':[],
               'arrival_gate':[],
               'arrival_time':[],
               'flight_status':[]
               }

for info in flight_response['arrivals']:
    flight_info['flight_number'].append(info.get('number', None)),
    flight_info['departure_airpot_name'].append(info['departure']['airport'].get('name',None)),
    flight_info['arrival_ICAO'].append(ICAO),
    flight_info['arrival_terminal'].append(info['arrival'].get('terminal', None)),
    flight_info['arrival_gate'].append(info['arrival'].get('gate', None)),
    flight_info['arrival_time'].append(info['arrival']['scheduledTime'].get('local', None)),
    flight_info['flight_status'].append(info.get('status',None))
    
    

flight_info = pd.DataFrame(flight_info)
                                                                          

In [94]:
flight_info['arrival_time']= flight_info['arrival_time'].str[:16] #we can also use [-6] to remove the last six characters

In [50]:
flight_info

,flight_number,departure_airpot_name,arrival_ICAO,arrival_terminal,arrival_gate,arrival_time,flight_status
0,FR 1143,Lisbon,EDDB,2,Z34,2024-05-08 11:10,Arrived
1,AY 1433,Helsinki,EDDB,1,A36,2024-05-08 12:25,Arrived
2,EC 5064,Milan,EDDB,1,A32,2024-05-08 12:40,Arrived
3,EW 8601,Porto,EDDB,1,A31,2024-05-08 12:35,Arrived
4,FR 1508,Castellón de la Plana,EDDB,2,Z36,2024-05-08 12:15,Arrived
...,...,...,...,...,...,...,...
225,FR 5418,Dublin,EDDB,2,Z43,2024-05-08 23:00,Expected
226,I2 3674,Madrid,EDDB,1,A32,2024-05-08 23:00,Expected
227,IB 3674,Madrid,EDDB,1,None,2024-05-08 23:00,Expected
228,SR 6493,Hurghada,EDDB,1,N01,2024-05-08 23:25,Expected


#### Looping for Both Countries

In [75]:
ICAO = ['EDDB', 'EGLL']
berlin_timezone = timezone('Europe/Berlin')
timestamp= datetime.now(berlin_timezone).strftime("%Y-%m-%d")
flight_info = {'flight_number':[],
             'departure_airpot_name': [],
             'arrival_ICAO':[],
             'arrival_terminal':[],
             'arrival_gate':[],
             'arrival_time':[],
             'flight_status':[]
                   }  

for icao in ICAO:
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{timestamp}T00:00/{timestamp}T12:00"
    querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
    headers = {
    	"X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
    	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    flight_response= response.json()
    for info in flight_response['arrivals']:
        flight_info['flight_number'].append(info.get('number', None)),
        flight_info['departure_airpot_name'].append(info['departure']['airport'].get('name',None)),
        flight_info['arrival_ICAO'].append(icao),
        flight_info['arrival_terminal'].append(info['arrival'].get('terminal', None)),
        flight_info['arrival_gate'].append(info['arrival'].get('gate', None)),
        flight_info['arrival_time'].append(info['arrival']['scheduledTime'].get('local', None)),
        flight_info['flight_status'].append(info.get('status',None))
        
        
    
flight_info = pd.DataFrame(flight_info)
flight_info['arrival_time']= flight_info['arrival_time'].str[:16]



#### Placing Data in a Function

In [77]:
airport_df

,icao,city_id
0,EDDB,1
1,EGLL,2


In [93]:
def get_flight_info (airport_df):
    
    now = datetime.now()
    tomorrow = now + timedelta(days=1)
    tomorrow = tomorrow.strftime('%Y-%m-%d')

    times_list = [{'from':'00:00','to':'11:59'},{'from':'12:00','to':'23:59'}]
    flight_info = {
                 'flight_number':[],
                 'departure_airpot_name': [],
                 'arrival_ICAO':[],
                 'arrival_terminal':[],
                 'arrival_gate':[],
                 'arrival_time':[],
                 'flight_status':[]
                       }  
    for i,row in airport_df.iterrows():
        for time in times_list:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{row['icao']}/{tomorrow}T{time['from']}/{tomorrow}T{time['to']}"
            querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
            headers = {
            	"X-RapidAPI-Key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
            	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            response = requests.get(url, headers=headers, params=querystring)
            flight_response= response.json()
            for info in flight_response['arrivals']:
                
                flight_info['flight_number'].append(info.get('number', None)),
                flight_info['departure_airpot_name'].append(info['departure']['airport'].get('name',None)),
                flight_info['arrival_ICAO'].append(row['icao']),
                flight_info['arrival_terminal'].append(info['arrival'].get('terminal', None)),
                flight_info['arrival_gate'].append(info['arrival'].get('gate', None)),
                flight_info['arrival_time'].append(info['arrival']['scheduledTime'].get('local', None)),
                flight_info['flight_status'].append(info.get('status',None))
                    
    flight_info = pd.DataFrame(flight_info)
    flight_info['arrival_time']= flight_info['arrival_time'].str[:16]
    #flight_info.to_sql('flights',con=connect, if_exists='append', index=False )

    

In [96]:
#get_flight_info (airport_df)